# Chapter 8. 차원 축소

많은 경우 머신러닝 문제는 훈련 샘플 각각이 수천 심지어 수백만 개의 특성을 가지고 있습니다. 이는 훈련을 느리게 할 뿐만 아니라 앞으로 보게되겠지만 좋은 솔루션을 찾기 어렵게 만듭니다. 이런 문제를 종종 **차원의 저주**(curse of dimensionality)라고 합니다.

다행히도 실전 문제에서는 특성 수를 크게 줄여서 불가능한 문제를 가능한 범위로 변경할 수 있는 경우가 많습니다. 분명 차원을 축소시키면 일부 정보가 유실됩니다. (JPEG로 이미지를 압축하면 품질이 감소되는 것처럼) 그래서 훈련 속도가 빨라질 수는 있지만 시스템의 성능이 조금 나빠질 수 있습니다. 또한 직업 파이프라인이 조금 더 복잡하게 되고 유지 관리가 어려워집니다. 그러므로 차원 축소를 고려하기 전에 훈련이 너무 느린지 먼저 원본 데이터로 시스템을 훈련시켜봐야 합니다. 그러나 어떤 경우에는 훈련 데이터의 차원을 축소시키면 잡음이나 불필요한 세부 사항을 걸러내므로 성능을 높일 수 있습니다.

훈련 속도를 높이는 것 외에 차원 축소는 데이터 시각화에도 아주 유용합니다. 차원 수를 둘로 줄이면 고차원 훈련 세트를 하나의 그래프로 그릴 수 있고 군집 같은 시각적인 패턴을 감지해 중요한 통찰을 얻는 경우가 많습니다. 

이 장에서는 차원의 저주에 대해 논의하고 고차원의 공간에서 어떤 일이 일어나고 있는지 알아보도록 하겠습니다. 그런 다음 차원 축소에 사용되는 두 가지 주요 접근 방법을 소개하겠습니다. (투영(projection)과 매니폴드 학습(manifold learning)) 그리고 가장 인기있는 차원 축소 기법인 PCA, 커널 PCA, LLE를 다루도록 하겠습니다.

## 1. 차원의 저주

우리는 3차원의 세계에서 살고있어서 고차원의 공간을 직관적으로 상상하기 어렵습니다. 고차원에서는 많은 것이 상당히 다르게 작동합니다. 예를 들어 단위 면적 안에 있는 점을 무작위로 선택한다면 경계선에서 0.001 이내에 위치할 가능성은 0.4%입니다. 하지만 10,000차원의 단위 면적을 가진 초입방체에서는 이 가능성이 99.99999%보다 커집니다. 고차원 초입방체에 있는 대다수의 점은 경계와 매우 가까이 있습니다.

고차원의 데이터는 예측을 위해 많은 외삽(extrapolation)을 해야하기 때문에 저차원일때보다 예측이 더 불안정합니다. 간단히 말해 *훈련 세트의 차원이 클수록 과대 적합의 위험이 커집니다.*

이론적으로 차원의 저주를 해결하는 해결책 하나는 훈련 샘플의 밀도가 충분히 높아질 때 까지 훈련 세트의 크기를 키우는 것입니다. 불행하게도 실제로는 일정 밀도에 도달하기 위해 필요한 훈련 샘플 수는 차원 수가 커짐에 따라 기하급수적으로 늘어납니다. 특성 수가 단 100개라고 생각해도 모든 차원에 걸쳐 균일하게 퍼져있다고 가정하고 훈련 샘플을 서로 평균 0.1 이내에 위치시키려면 관측 가능한 우주에 있는 원자 수를 모두 합친 것보다 더 많은 훈련 샘플을 모아야 합니다. (오우..)



## 2. 차원 축소를 위한 접근 방법

구체적인 차원 축소 알고리즘을 알아보기 전에 차원을 감소시키는 두 가지 주요한 접근법인 투영(projection)과 매니폴드 학습(manifold learning)을 알아봅시다.

### 2.1 투영

대부분의 실전 문제는 훈련 샘플이 모든 차원에 걸쳐 균일하게 퍼져 있지 않습니다. 많은 특성은 거의 변화가 없는 반면, 다른 특성들은 서로 강하게 연관되어 있습니다. 결과적으로 모든 훈련 샘플이 사실 고차원 공간 안의 저차원 **부분 공간**(subspace)에 놓여있습니다. 

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig1.png)

모든 훈련 샘플이 거의 평면 형태로 놓여있습니다. 이것이 고차원(3D) 공간에 있는 저차원(2D) 부분 공간입니다. 여기서 모든 훈련 샘플을 이 부분 공간에 수직으로(즉, 샘플과 평면 사이의 가장 짧은 직선을 따라) 투영하면 아래 그림과 같은 2D 데이터셋을 얻습니다. 데이터 셋의 차원을 3D에서 2D로 줄인 것입니다. 각 축은 새로운 특성 $z_1$과 $z_2$에 대응됩니다.

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig2.png)

그러나 차원 축소에 있어 투영이 언제나 최선의 방법은 아닙니다. 많은 경우 아래 그림처럼 **스위스 롤**(swiss roll) 데이터셋처럼 부분 공간이 뒤틀리거나 휘어 있기도 합니다. 

![img](https://unsolvedproblem.github.io/assets/images/Hands-on/Ch8fig3.png)